In [1]:
# !pip install torchmetrics


In [2]:
# !pip install lightning

In [3]:
import random
import numpy as np
import os
import torch
from torch import optim
from torch.utils.data import Dataset, random_split, DataLoader
from torchmetrics import Accuracy, F1Score, AUROC, Precision
from torchvision import transforms
from torchvision.models import VGG16_Weights, ResNet18_Weights
from torch.nn import functional as F
import lightning.pytorch as pl
from lightning.pytorch import loggers as pl_loggers
from lightning.pytorch.loggers import TensorBoardLogger
import torchvision.models
from PIL import Image
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint


In [4]:
def get_transformation(size=128):
    return transforms.Compose([
        transforms.Resize(size=(size, size)),
        transforms.RandomRotation(degrees=(-45, 45)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.RandomApply(torch.nn.ModuleList([transforms.GaussianBlur(kernel_size=3)]), p=0.5),
        transforms.RandomApply(
            torch.nn.ModuleList([transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.2, hue=0.2)]),
            p=0.5),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

    ])


In [5]:
def get_test_transformation(size=224):
    return transforms.Compose([
        transforms.Resize(size=(size, size)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])


In [6]:
class AnimalImage(Dataset):
    def __init__(self, data_root, transformation, mode="train"):
        self.data_root = data_root
        self.transformation = transformation
        self.list_imgs = os.listdir(self.data_root)
        self.mode = mode

    def __len__(self):
        return len(self.list_imgs)

    def __getitem__(self, idx):
        img_name = self.list_imgs[idx]
        img = self.load_img(img_path=self.data_root + "/" + img_name)
        img = self.transformation(img)
        label = 0  # if it's a dog then label = 0
        if "cat" in img_name:
            label = 1

        return {"img": img, "label": label}

    def load_img(self, img_path):
        img = Image.open(img_path)
        return img


In [7]:
class Model(pl.LightningModule):
    def __init__(self, model, lr, wd):
        super().__init__()
        # Define metrics for binary classification
        self.initialize_metrics()
        self.model = model
        self.model.fc = torch.nn.Sequential(
            model.fc,
            torch.nn.Linear(1000, 1),
        )

        self.lr = lr
        self.wd = wd

    def initialize_metrics(self):
        task = "binary"
        self.metrics_list = ["accuracy", "precision", "f1", "auc"]
        self.sessions = ["train", "val", "test"]
        self.classes = [("cat", 1), ("dog", 0)]

        self.train_ac = Accuracy(task=task, average="macro")
        self.val_ac = Accuracy(task=task, average="macro")
        self.test_ac = Accuracy(task=task, average="macro")

        self.train_p = Precision(task=task, average="macro")
        self.val_p = Precision(task=task, average="macro")
        self.test_p = Precision(task=task, average="macro")

        self.train_f1 = F1Score(task=task, average="macro")
        self.val_f1 = F1Score(task=task, average="macro")
        self.test_f1 = F1Score(task=task, average="macro")

        self.train_auc = AUROC(task=task, average="macro")
        self.val_auc = AUROC(task=task, average="macro")
        self.test_auc = AUROC(task=task, average="macro")

        self.metrics = {"train": [self.train_ac, self.train_p, self.train_f1, self.train_auc],
                        "val": [self.val_ac, self.val_p, self.val_f1, self.val_auc],
                        "test": [self.test_ac, self.test_p, self.test_f1, self.test_auc],
                        }
        self.step_output = {"train": [], "val": [], "test": []}
        self.mean_log_keys = ["loss"]

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=self.lr, weight_decay=self.wd)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=1e-6)
        return [optimizer], [scheduler]

    def forward(self, x):
        x = self.model(x)
        return x

    # Calculate the loss for the given batch
    def _calculate_loss(self, batch):
        imgs, labels = batch["img"], batch["label"].float().unsqueeze(1)
        preds = self.forward(imgs)
        loss = F.binary_cross_entropy_with_logits(preds, labels)
        return {"loss": loss, "preds": preds, "labels": labels}

    # For model A
          # Training step
    def training_step(self, batch, batch_idx):
        output = self._calculate_loss(batch)
        self.step_output["train"].append(output)
        return output["loss"]
    def on_train_epoch_end(self):
        self.stack_update(session="train")

    # Validation:
    def validation_step(self, batch, batch_idx):
        output = self._calculate_loss(batch)
        self.step_output["val"].append(output)
        return output["loss"]
    def on_validation_epoch_end(self):
        self.stack_update(session="val")

    # Test:
    def test_step(self, batch, batch_idx):
        output = self._calculate_loss(batch)
        self.step_output["test"].append(output)
        return output["loss"]

    # Actions to perform at the end of each test epoch
    def on_test_epoch_end(self, ):
        self.stack_update(session="test")

    # Update metrics with predictions and labels
    def update_metrics(self, session, preds, labels):
        for metric in self.metrics[session]:
            metric.update(preds, labels)
            
    # Preparation for Model B: SRGAN model
    def stack_update(self, session):
        all_preds = torch.cat([out["preds"] for out in self.step_output[session]])
        all_labels = torch.cat([out["labels"] for out in self.step_output[session]])
        log = {}
        for key in self.mean_log_keys:
            log[f"{session}_{key}"] = torch.stack([out[key] for out in self.step_output[session]]).mean()

        self.update_metrics(session=session, preds=all_preds, labels=all_labels)
        res = self.compute_metrics(session=session)
        self.add_log(session, res, log)
        self.log_dict(log, sync_dist=True, on_epoch=True, prog_bar=True, logger=True)
        self.restart_metrics(session=session)

        return all_preds, all_labels

    def compute_metrics(self, session):
        res = {}
        for metric, metric_name in zip(self.metrics[session], self.metrics_list):
            res[metric_name] = metric.compute()
        return res

    def restart_metrics(self, session):
        for metric in self.metrics[session]:
            metric.reset()
        self.step_output[session].clear()  # free memory

    # Metrics to the SRGAN_train.log
    
    def add_log(self, session, res, log):
        for metric in self.metrics_list:
            log[session + f"_{metric}"] = res[metric]



In [8]:
def set_seed(seed=10):
    pl.seed_everything(seed)
    np.random.seed(seed=seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.random.manual_seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.set_float32_matmul_precision('medium')


In [9]:
if __name__ == "__main__":
    # Load the resnet model with pre-trained weights
    model_architecture = torchvision.models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
    model = Model(model=model_architecture, lr=1e-4, wd=1e-6)

    max_epochs = 100
    torch.set_float32_matmul_precision('medium')
    model_path = "checkpoints/Model_A"
    batch_size = 32
    # Set seed for reproducibility
    seed = 10
    set_seed(seed)
    # Create dataset and split it into train, and validation
    cat_dog_dataset = AnimalImage(data_root="/home/mdnurualabsarsiddiky/Desktop/Absar/ECGR8119/Midterm_2024/dogs_vs_cats/train", transformation=get_transformation())
    train_data, val_data, _ = random_split(cat_dog_dataset, [0.6, 0.1, 0.3],
                                           generator=torch.Generator().manual_seed(seed))
    ##
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True, pin_memory=True, num_workers=2)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=2, drop_last=False)

    # Set up logging for training progress
    csv_logger = pl_loggers.CSVLogger(save_dir=os.path.join(model_path, "SRGAN/SRGAN_train.log/"))
    tb_logger = TensorBoardLogger(save_dir=os.path.join(model_path, "tb_log/"), name="resnet18")
    # Define early stopping criteria
    monitor = "val_loss"
    mode = "min"
    early_stopping = EarlyStopping(monitor=monitor, patience=10, verbose=False, mode=mode)
    # Initialize the trainer and start training
    trainer = pl.Trainer(
        default_root_dir=model_path,
        accelerator="gpu",
        max_epochs=max_epochs,
        callbacks=[
            early_stopping,
            ModelCheckpoint(dirpath=model_path, filename="resnet18-{epoch}-{val_loss:.2f}", save_top_k=1,
                            save_weights_only=True, mode=mode, monitor=monitor),
        ],
        logger=[tb_logger, csv_logger],
    )
    trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)


Seed set to 10
Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Missing logger folder: checkpoints/Model_A/tb_log/resnet18
/home/mdnurualabsarsiddiky/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/mdnurualabsarsiddiky/.local/lib/python3.8/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:652: Checkpoint directory /home/mdnurualabsarsiddiky/Desktop/Absar/ECGR8119/Midterm_2024/Model_A/checkpoints/Model_A exists and is not empty.
LOCAL

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]